In [9]:
import os, json

with open('conf.json', 'r') as f:
        json_data = json.load(f)
    
os.environ['OPENAI_API_KEY'] = json_data['API_KEY']

In [10]:
from core.get_common_issues import get_news_dict_list

etf_tkr = "AIQ"

news_data = str(get_news_dict_list(etf_tkr=etf_tkr))

In [11]:
from data.fetch_data import fetch_data_from_db

import pandas as pd

sql = """
    select * from os_index_info
"""

etf_df = pd.DataFrame(fetch_data_from_db(query=sql))

etf_df.head()

,etf_tkr,index_name,upper_bound,extra_info,lower_bound,rebalance_period
0,AGNG,Indxx Aging Population Thematic Index,0.03,"3% 이상의 가중치를 가진 보안은 3%로 할당되고, 0.3% 이하의 가중치를 가진 ...",0.003,인덱스는 매년 한 번의 재구성과 재조정 일정을 따릅니다. 새로운 포트폴리오는 매년 ...
1,AIQ,Indxx Artificial Intelligence & Big Data Index,0.03,초과 가중치는 남은 제한이 없는 보안 중에서 비례하여 분배. 노출 점수가 20% 이...,0.001,\n\n인덱스는 매년 한 번의 재구성과 재조정 일정을 따릅니다. 새로운 포트폴리오는...
2,AQWA,Solactive Global Clean Water Industry Index,0.07,"7% 이상의 가중치를 가진 지수 구성요소의 총합 가중치는 40%로 제한, 나머지 종...",0.003,.
3,BFIT,Indxx Global Health & Wellness Thematic Index,0.03,"3% 이상의 가중치를 가진 보안은 3%로 할당되고, 0.3% 이하의 가중치를 가진 ...",0.003,인덱스는 매년 두 번의 재구성과 재조정 일정을 따릅니다. 인덱스는 매년 4월과 10...
4,BKCH,Solactive Blockchain Index,0.12,"4.5% 이상의 가중치를 가진 지수 구성요소의 총합 가중치는 45%로 제한, 나머지...",0.001,.


In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI

from tqdm.auto import tqdm

def get_llm_answer(etf_tkr, text):
    template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=(
                    f"""
                    Summarize the common issues based on the news headlines of stocks which are best 5 influential components of {etf_tkr} in Korean.
                    
                    - Tone : Humble
                    - Style : MECE, accurate
                    - Reader level : college student
                    - Length : within 500 characters
                    - Perspective : Mckinsey&Company senior consultant
                    - Format : markdown
                    
                    You MUST answer in Korean. 
                    """
                )
            ),
            HumanMessagePromptTemplate.from_template("{text}"),
        ]
    )

    llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
    
    ret = llm(template.format_messages(text=text))

    return ret

for i, row in tqdm(etf_df.iterrows()):
    
    etf_tkr = row.etf_tkr
    
    etf_name = row.index_name
    
    news_data_per_etf = str(get_news_dict_list(etf_tkr=etf_tkr))

    llm_answer = get_llm_answer(etf_tkr=etf_name, text=news_data_per_etf)

    print(llm_answer.content)
    
    print()

    with open(f'./etf_llm_summary/{etf_tkr}.json', 'w') as outfile:
        json.dump({"llm_answer": llm_answer.content}, outfile, indent='\t', ensure_ascii=False)

0it [00:00, ?it/s]

Indxx Aging Population Thematic Index의 주요 구성 요소 중에서 얻은 뉴스 헤드라인을 기반으로 일반적인 문제점을 요약해 보겠습니다.

1. Astrazeneca (AZN): 시장 대비 성장하였으나, 성장률이 낮음.
2. Johnson & Johnson (JNJ): 헤지 펀드에 따르면 하락장에서도 투자가치가 있는 주식 중 하나. 그러나 차트 분석 결과는 좋지 않음.
3. Bristol-Myers Squibb (BMY): 지난 1년간 주식 투자로 17% 손실을 입음. 그러나 미국 의료보험 가격 협상에 참여하는 10개의 의약품 회사 중 하나.
4. AbbVie (ABBV): 67%의 ROE로 인상적인 수익률을 보여줌. 높은 배당 주식 중 하나.
5. Boston Scientific (BSX): 내재 가치 계산에 따르면 가치가 있음. 의료 주식 중에서도 성장률이 높음.

이러한 문제점을 고려하여 Indxx Aging Population Thematic Index의 구성 요소인 주식들의 투자 가능성을 평가할 수 있습니다.

인덱스에 포함된 주식들의 뉴스 헤드라인을 기반으로 한 공통된 이슈는 다음과 같습니다.

1. 스마트폰 산업의 침체와 인공지능의 치료: 스마트폰 시장이 침체 상태에 빠져 있으며, 인공지능이 이를 치료할 수 있는 가능성이 제기되고 있습니다.

2. 인공지능 기술의 발전과 주식 시장: 인공지능 기술의 발전에 따라 인공지능 관련 주식들의 가치가 상승하고 있으며, 헤지 펀드들이 주식 구매를 권장하고 있습니다.

3. 구글과 애플의 법규 위반 문제: 구글과 애플이 한국의 앱 내 빌링 규정을 위반하여 벌금을 부과받을 가능성이 있습니다.

4. 아마존의 다양한 이슈: 아마존은 휴일에는 Whole Foods에서 쇼핑을 자제할 것을 권장하고 있으며, 위성 인터넷 시장에 도전하기 위해 첫 위성을 발사하였습니다.

5. 마이크로소프트의 소프트웨어 주식과 투자: 소프트웨어 주식 시장이 침체 상태에 빠져 있으며, 마이크로소프트의 주식 가치에 대한